In [112]:
from dotenv import load_dotenv
from hpack.hpack import INDEX_NEVER
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_pinecone import PineconeVectorStore

load_dotenv()

True

## 파일로드

In [21]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import PyPDFLoader

dur_pregnant = CSVLoader('./data/DUR_pregnant.csv')
commandment_10 = PyPDFLoader('./data/command_10.pdf')
docs = PyPDFLoader('./data/docs.pdf')
drug_for = PyPDFLoader('./data/drug_for.pdf')

dur_pregnant_docs = dur_pregnant.load()
commandment_10_docs = commandment_10.load()
docs_docs = docs.load()
drug_for.docs = drug_for.load()

In [22]:
documents = commandment_10_docs + docs_docs + drug_for.docs

In [23]:
# 개행 문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
documents

[Document(metadata={'source': './data/command_10.pdf', 'page': 0}, page_content='■ 임산부의 안전하고, 효과적인 약 복용을 위한 약물 요법 10계명    1. 임신 가능한 여성의 경우 약물 복용 전 임신 여부 확인하십시오. 임신 가능한 여 성이 약물을 복용할 때는 임신 계획 및 임신 여부를 반드시 확인하여야 하며 전 문가와 상의 후 투여하는 것이 필요합니다.  2. 임신 중 약물 복용은 태아에게 영향을 줄 수 있습니다. 임산부가 약물 복용 시  주의해야 하는 까닭은 약물이 태반을 통과하여 태아에게 나쁜 영향을 미칠 수  있고 임산부에게도 독성을 일으킬 수 있기 때문입니다  3. 임신 중 약물 복용 방법은 일반 성인과는 다를 수 있습니다 임신 중에는 복용한  약물이 태반을 통과할 수 있다는 것을 항상 염두 해두셔야 하며 임신 중 약물의  체내 움직임은 일반 성인과는 다른 형태로 바뀌어 나타나므로 약물의 복용량과  복용법 변화에 유의하셔야 합니다.  4. 임신 3주에서 8주 사이 약물 복용 시 더욱 주의하셔야 합니다. 임신 주수에 따라  약물이 태아에게 미치는 영향이 다양합니다. 특히, 임신 3주에서 8주 사이 (임신  제 1 삼분기)에는 태아의 장기가 형성되는 결정적인 시기이므로 특별히 조심해야  합니다.  5. 약물마다 임신 시 미치는 영향에 따른 카테고리가 구분되어 있습니다. 미국 FDA  에서는 약물마다 태아에 미치는 영향을 가장 안전한 카테고리 A부터 가장 위험 한 카테고리 X까지 다섯 단계로 구분하여 표시하고 있으므로 그 위험성을 예측 할 수 있습니다.  6. 임신 중 약물 투여는 반드시 전문가 (의약사)와 상의 후 투여하십시오. 임신 중에  약물을 투여할 경우, 약물치료가 꼭 필요한지 전문가와 다시 검토하고 대증요법  등의 다른 치료방법을 우선적으로 모색해야 합니다.  7. 임신 중 질환에 따라 약물을 꼭 복용해야 하는 경우도 있습니다 임신 중 간질,  고혈압, 당뇨 등의 질환을 동반하는

## Text Spliter

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

docs = spliter.split_documents(documents)
print(len(docs))
docs

41


[Document(metadata={'source': './data/command_10.pdf', 'page': 0}, page_content='■ 임산부의 안전하고, 효과적인 약 복용을 위한 약물 요법 10계명    1. 임신 가능한 여성의 경우 약물 복용 전 임신 여부 확인하십시오. 임신 가능한 여 성이 약물을 복용할 때는 임신 계획 및 임신 여부를 반드시 확인하여야 하며 전 문가와 상의 후 투여하는 것이 필요합니다.  2. 임신 중 약물 복용은 태아에게 영향을 줄 수 있습니다. 임산부가 약물 복용 시  주의해야 하는 까닭은 약물이 태반을 통과하여 태아에게 나쁜 영향을 미칠 수  있고 임산부에게도 독성을 일으킬 수 있기 때문입니다  3. 임신 중 약물 복용 방법은 일반 성인과는 다를 수 있습니다 임신 중에는 복용한  약물이 태반을 통과할 수 있다는 것을 항상 염두 해두셔야 하며 임신 중 약물의  체내 움직임은 일반 성인과는 다른 형태로 바뀌어 나타나므로 약물의 복용량과  복용법 변화에 유의하셔야 합니다.  4. 임신 3주에서 8주 사이 약물 복용 시 더욱 주의하셔야 합니다. 임신 주수에 따라  약물이 태아에게 미치는'),
 Document(metadata={'source': './data/command_10.pdf', 'page': 0}, page_content='나타나므로 약물의 복용량과  복용법 변화에 유의하셔야 합니다.  4. 임신 3주에서 8주 사이 약물 복용 시 더욱 주의하셔야 합니다. 임신 주수에 따라  약물이 태아에게 미치는 영향이 다양합니다. 특히, 임신 3주에서 8주 사이 (임신  제 1 삼분기)에는 태아의 장기가 형성되는 결정적인 시기이므로 특별히 조심해야  합니다.  5. 약물마다 임신 시 미치는 영향에 따른 카테고리가 구분되어 있습니다. 미국 FDA  에서는 약물마다 태아에 미치는 영향을 가장 안전한 카테고리 A부터 가장 위험 한 카테고리 X까지 다섯 단계로 구분하여 표시하고 있으므로 그 위험성을 예측 할 수 있습니다.  6. 임신 중


# 임베딩 모델 & Pinecone

In [25]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)

In [26]:
import os

PINECONE_INDEX_NAME1 = os.getenv('PINECONE_INDEX_NAME1')
PINECONE_NAMESPACE1 = os.getenv('PINECONE_NAMESPACE1')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [27]:
print(PINECONE_INDEX_NAME1)
print(PINECONE_NAMESPACE1)
print(PINECONE_API_KEY)

ai-medicine-docs
ai-medicine-docs-ns
pcsk_4XyuMX_5f47dcAxtKB3mkLzQbc4EfucgVHPSkZ7pikmbjCRbviWzyUBGstWunj4XTVTmaj


In [28]:
from langchain_pinecone import PineconeVectorStore
vector_db1 = PineconeVectorStore.from_documents(
    docs,
    embeddings,
    index_name=PINECONE_INDEX_NAME1,
    namespace=PINECONE_NAMESPACE1,
    pinecone_api_key=PINECONE_API_KEY
)

In [35]:
vector_db1 = PineconeVectorStore(
    embedding=embeddings,
    index_name=PINECONE_INDEX_NAME1,
    namespace=PINECONE_NAMESPACE1,
    pinecone_api_key=PINECONE_API_KEY
)

results = vector_db1.similarity_search(
    "임산부가 주의해야할 약물에 대해 말해줘",
    k=3, # 유사도 높은 순으로 5건
    namespace=PINECONE_NAMESPACE1, # 검색의 논리적인 단위
)

In [87]:
page_content = [doc.page_content for doc in results]
page_content

['임산부의 약물 상담 지침 그림 1. 임신중 약물의 위험성. 약물들의 대부분은 대기형발생위험률이 기본위험률(1-3%)을 넘지 않는다. 단지 몇 가지 약물만이 20~30%의 위험률을 가짐. FDA 분류를 임신인지 모르고 약물에 노출된 임신부에게 약물의 안전성 및 위험성 평가를 위해 적용하는 것은 옳은가? 옳지 않다. 미국 FDA 분류는 1979년에 임신부의 치료지침을 제공하기 위해 개발되었지만, 이상적이지 않은 것으로 평가되고 있다. 대부분의 약물의 분류가 동물실험자료, 증례발표, 그리고 사람에 대한 자료가 제한되어 있거나 거의 없는 상태에 근거하고 있다. 그리고 최근 약물의 안전성과 위험성에 관한 증거가 있음에도 잘 반영되지 않고 있다. 또한, 같은 군에 속해 있는 경우 유사한 위험을 갖는 것으로 인식하게 한다거나, D군이나 X군 그리고 심지어 C군에 포함되는 약물들도 유사한 위험을 가질 수 있어서 의료인들이 약물을 해석하는데 혼 란을 가져올 수 있다. 따라서, FDA는 이러한',
 '임산부의 약물 상담 지침 그림 1. 임신중 약물의 위험성. 약물들의 대부분은 대기형발생위험률이 기본위험률(1-3%)을 넘지 않는다. 단지 몇 가지 약물만이 20~30%의 위험률을 가짐. FDA 분류를 임신인지 모르고 약물에 노출된 임신부에게 약물의 안전성 및 위험성 평가를 위해 적용하는 것은 옳은가? 옳지 않다. 미국 FDA 분류는 1979년에 임신부의 치료지침을 제공하기 위해 개발되었지만, 이상적이지 않은 것으로 평가되고 있다. 대부분의 약물의 분류가 동물실험자료, 증례발표, 그리고 사람에 대한 자료가 제한되어 있거나 거의 없는 상태에 근거하고 있다. 그리고 최근 약물의 안전성과 위험성에 관한 증거가 있음에도 잘 반영되지 않고 있다. 또한, 같은 군에 속해 있는 경우 유사한 위험을 갖는 것으로 인식하게 한다거나, D군이나 X군 그리고 심지어 C군에 포함되는 약물들도 유사한 위험을 가질 수 있어서 의료인들이 약물을 해석하는데 혼 란을 가져올 수 있다. 따라서, FDA는 이러한'

## prompt

In [138]:
prompt = ChatPromptTemplate.from_messages([
        ('system', '''
Persona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.

Role: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.

Examples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.
        '''),
        ('user', '''
        query 임산부가 복용해도 되는 약물에 대해 context만 참조해서 알려줘
        
        question: {query}
        context: {context}
        ''')
        
    ]) 
prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nPersona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.\n\nExamples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.\n        '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\n        query 임산부가 복용해도 되는 약물에 대해 context만 참조해서 알려줘\n        \n        question: {query}\n        context: {context}\n        '), additional_kwargs={})])

## PDF_retriever_vector_store

In [139]:
retriever = vector_db1.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':3}
)
query ='임산부에 약물 복용'

retrievals = retriever.batch([query])

In [102]:
retrievals

[[Document(id='b3ea4e13-6fe3-47d3-99cc-6b514994a964', metadata={'page': 9.0, 'source': './data/korean_mothersafe.pdf'}, page_content='임산부의 약물 상담 지침 8 감기, 근육통 등으로 약물복용 시 모유수유 중단해야 하나요? 모유는 영양적, 면역학적 그리고 정서적으로 아기에게 가장 적합한 영양소이다. 따라서, 비록 여러 가지 질병으로 치료를 위해 약물을 복용해야 하는 경우도 모유수유를 중단하는 것은 신중하게 결정해야 한다. 알려진 바로는 아기에게 전달되는 약물의 양은 엄마가 복용하는 양의 1-2% 뿐으로 임상적으로 아기에게 미치는 영향이 거의 없어서 감기, 근육통에 처방하는 약물을 포함해 대부분의 약물들은 모유수유에 적합한 것으로 알려져 있다. 현재까지 알려진 바로는 메토트렉세이트는 주요기형유발물질로 임신 초기에 사용시 5% 정도에서 무뇌아, 두개골형성장애, 두개결합증 등을 유발할 수 있다. 이 약의 경우 얼마나 중단 후 임신 할 수 있는지에 대하여 일치된 의견이 있지는 않지만 체내에서 이 약물이 제거되는 시간을 고려하면 3개월 정도 지난 후에는 임신이 가능해 보인다. 그리고 나머지 소염진통제, 코르티코스테로이드,'),
  Document(id='5e6414a1-7c5d-4e75-88d8-06eb75ede309', metadata={'page': 9.0, 'source': './data/korean_mothersafe.pdf'}, page_content='임산부의 약물 상담 지침 8 감기, 근육통 등으로 약물복용 시 모유수유 중단해야 하나요? 모유는 영양적, 면역학적 그리고 정서적으로 아기에게 가장 적합한 영양소이다. 따라서, 비록 여러 가지 질병으로 치료를 위해 약물을 복용해야 하는 경우도 모유수유를 중단하는 것은 신중하게 결정해야 한다. 알려진 바로는 아기에게 전달되는 약물의 양은 엄마가 복용하는 양의 1-2% 뿐으로 임상적으로 아기에게 미치는 영향이 거

In [140]:
prompt.invoke({'query':query, 'context':retriever})

ChatPromptValue(messages=[SystemMessage(content='\nPersona: 당신은 전문적인 약사입니다. 약에 대해 복용 방법, 주의 사항, 금기 여부를 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 임산부에게 의약품의 복용 가능 여부 및 금기 등급, 금기 등급의 의미를 정확히 이해할 수 있도록 돕습니다. context를 기반으로 약품에 대해 아무것도 모르는 임산부가 이해할 수 있도록 주의 사항을 도출해서 5줄 이내로 알려줍니다.\n\nExamples: 만약 해당 성분이 임산부에게 해로우면, 해당 의약품이 어떤 성분으로 인해 어느 금기 등급에 속하며 어떤 위험성이 있는지 알려준다. 만약 해당 성분이 임산부에게 해롭지 않으면, DUR 임부금기에 해당되지 않으므로 복용해도 된다고 알려준다.\n        ', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n        query 임산부가 복용해도 되는 약물에 대해 context만 참조해서 알려줘\n        \n        question: 임산부에 약물 복용\n        context: tags=['PineconeVectorStore', 'OpenAIEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000020DA4836D80> search_kwargs={'k': 3}\n        ", additional_kwargs={}, response_metadata={})])

In [141]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o',
    temperature=0.5
)

In [142]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

re_chain = {'query': RunnablePassthrough(), 'context':retriever} | prompt | model | StrOutputParser()

re_chain.invoke('query')

'임산부가 복용하면 안 되는 약물로는 Diethylstilbestrol (DES), Aminopterin, Methotrexate, Isotretinoin, Thalidomide, Misoprostol 등이 있습니다. 이러한 약물들은 임신 중 복용 시 태아에게 해로운 영향을 미칠 수 있으며, 기형을 유발할 수 있는 위험이 있습니다. 특히 이 약물들은 태아에게 직접적인 손상을 줄 수 있는 테라토겐으로 분류되며, 임산부에게 금기입니다. 임신 중에는 반드시 의사나 약사와 상담하여 약물 복용 여부를 결정하는 것이 중요합니다. DUR 임부금기에 해당되지 않는 약물은 의사의 지시에 따라 복용할 수 있습니다.'

In [143]:
re_chain.invoke('씨앤유캡슐')

'씨앤유캡슐에 대한 정보는 주어진 context에 명시되어 있지 않습니다. 그러나, 임산부가 복용 가능한 약물로는 아세트아미노펜, 페니실린 계열, 세팔로스포린 계열, 마크로라이드 계열, 메트로니다졸 등이 있습니다. 이 약물들은 일반적으로 임산부에게 안전하게 사용될 수 있지만, 의사나 약사와 상담 후 사용하는 것이 좋습니다. 씨앤유캡슐이 어떤 성분으로 구성되어 있는지를 확인하고, 그 성분이 임산부에게 안전한지를 전문가와 상담하는 것이 필요합니다.'

## simirality_search

In [114]:
import os
PINECONE_INDEX_NAME2 = os.getenv('PINECONE_INDEX_NAME2')
PINECONE_NAMESPACE2 = os.getenv('PINECONE_NAMESPACE2')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

print(PINECONE_INDEX_NAME2)
print(PINECONE_NAMESPACE2)
print(PINECONE_API_KEY)

ai-pregnant
ai-pregnant-ns
pcsk_4XyuMX_5f47dcAxtKB3mkLzQbc4EfucgVHPSkZ7pikmbjCRbviWzyUBGstWunj4XTVTmaj


In [116]:
def similarity_search_pregnant(query):
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME2,
        namespace=PINECONE_NAMESPACE2,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    result = vector_db.similarity_search(
        query,
        k=3,
        namespace=PINECONE_NAMESPACE2,
    )
    
    return{
        "query": query,
        "content" : '\n'.join([doc.page_content for doc in result]),
    }

In [124]:
simi_chain = RunnableLambda(similarity_search_pregnant)

print(simi_chain.invoke('씨앤유캡슐'))

{'query': '씨앤유캡슐', 'content': '성분명: Chenodeoxycholic acid + Ursodeoxycholic acid\n제품명: 씨앤유캡슐(케노데속시콜산-우르소데속시콜산삼수화물마그네슘염)_(0.25g/1캡슐)\n업체명: 명문제약(주)\n금기등급: 1\n상세정보: 임부 투여금기, 동물실험에서 간독성의 증거 나타남\n성분명: calcium phosphate + ascorbic acid + calcium pantothenate + cholecalciferol + cupric oxide + cyanocobalamin + ferrous sulfate + magnesium oxide + manganese + nicotinamide + pyridoxine + retinol palmitate + riboflavin   + thiamine + tocopherol +zinc oxide\n제품명: 비타헬스에이연질캡슐_(1캡슐)\n업체명: (주)알피바이오\n금기등급: 1\n상세정보: (retinol)임신초기 3개월간 고용량 투여시 최기형 가능성. (단, VitA(레티놀)로서 5000 I.U/1일 이상)\n성분명: dutasteride\n제품명: 유로다트연질캡슐0.5mg(두타스테리드)_(0.5mg/1캡슐)\n업체명: (주)씨티씨바이오\n금기등급: 1\n상세정보: 전임상시험에서 남성 태아의 외부 생식기 발달 저해 가능성.'}


In [132]:
chain =re_chain | simi_chain

In [144]:
print(chain.invoke('임신 계획중인데 씨앤유캡슐 복용 가능해?'))

{'query': '씨앤유캡슐에 대한 정보는 주어진 context에 포함되어 있지 않습니다. 일반적으로 임신 계획 중이거나 임신 중인 경우 어떤 약물이든 복용하기 전에 반드시 의사와 상의하는 것이 중요합니다. 임신 초기에는 특히 약물 복용에 주의가 필요하며, 기형 유발 가능성이 있는 약물은 피하는 것이 좋습니다. 씨앤유캡슐에 대한 구체적인 정보는 의사나 약사와 상담하여 안전성을 확인하시기 바랍니다.', 'content': '성분명: fluconazole\n제품명: 프리졸캡슐(플루코나졸)_(50mg/1캡슐)\n업체명: 주식회사 더유제약\n금기등급: 2\n상세정보: 임신 첫 3개월 안에 150mg을 단회 또는 반복 투여한 임부에서 자연유산 및 영아의 선천성이상 보고. 임신 첫 3개월의 모든 또는 대부분의 기간동안 고용량(1일 400-800mg) 투여 시 영아에서 출생기형(납작머리증, 비정상적 얼굴, 비정상정 머리덮개뼈 발달, 구개열, 대퇴부 휨, 얇은 갈비뼈, 긴뼈, 관절굽음증, 선천적 심장질환)이 나타남. 콕시디오이데스진균증에 대해 3개월 이상 고용량(1일 400-800mg) 투여 시 영아에서 다발성 선천성 기형 보고. 동물실험에서 고용량 투여 시 태자의 해부학적인 변형(과잉늑골, 신우확장) 증가 및 골화 지연, 배자치사 증가, 태아 기형(파형늑골, 구개열, 비정상적인 두개안면부 골화) 확인.\n성분명: fluconazole\n제품명: 푸가졸캡슐(플루코나졸)_(50mg/1캡슐)\n업체명: 신풍제약(주)\n금기등급: 2\n상세정보: 임신 첫 3개월 안에 150mg을 단회 또는 반복 투여한 임부에서 자연유산 및 영아의 선천성이상 보고. 임신 첫 3개월의 모든 또는 대부분의 기간동안 고용량(1일 400-800mg) 투여 시 영아에서 출생기형(납작머리증, 비정상적 얼굴, 비정상정 머리덮개뼈 발달, 구개열, 대퇴부 휨, 얇은 갈비뼈, 긴뼈, 관절굽음증, 선천적 심장질환)이 나타남. 콕시디오이데스진균증에 대해 3개월 이상 고용량(1일 400-800mg) 투여 시 영아에서 